# Dependências

In [2]:
#!pip uninstall psycopg2
#!pip list
#!pip install wheel
#!pip install setuptools
#!pip install psycopg2
#!pip install psycopg2-binary==2.8.3
#!pip3 install psycopg2
!pip install psycopg2-binary

In [3]:
!pip install pyarrow

# Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import psycopg2 as postgres
import math

# Funções

In [5]:
def connect():
    conn = None
    try:
        conn = postgres.connect(
            host="localhost",
            database="austin_test",
            user="postgres",
            password="root")
    except postgres.Error as e:
        print(e)
    return conn

In [6]:
def closeConnection(conn):
    sucess = False
    try:
        conn.close()
        sucess = True
    except postgres.Error as e:
        print(e)
    
    return sucess

In [7]:
def executeQuery(conn, sql):
    record = None
    try:
        #print(sql)
        cur = conn.cursor()
        cur.execute(sql)
        record = cur.fetchall()
        cur.close()
    except postgres.Error as e:
        print(e)
        cur.execute("ROLLBACK")
        cur.close()
    return record

In [8]:
def executeInsert(conn, sql):
    sucess = False
            
    try:
        #print(sql)
        cur = conn.cursor()
        cur.execute(sql)
        conn.commit()
        cur.close()
        sucess = True
    except postgres.Error as e:
        print(e)
        cur.execute("ROLLBACK")
        cur.close()

    return sucess

In [9]:
#Recebe um id e retorna as categorias e checkin do lugar
def getPOIInformation(conn, business_id,):
    
    sql = """
        SELECT checkin_count, name FROM pois_information WHERE id  = \'"""+str(business_id)+ """\'
    ;"""

    result = executeQuery(conn, sql)

    return result

In [10]:
#Encontra todos os pontos do bin centrado em um POI trazendo suas informações (categorias e checkin)
def getBinInformation(conn, business_id, bin_number):

    result = None

    sql = """
        SELECT fk_poi_id_context, name, checkin_count, distance_m 
        FROM bins_information 
        WHERE fk_poi_id_center = \'"""+str(business_id)+"""\' AND fk_bin_number = """+str(bin_number)+""";"""

    result = executeQuery(conn, sql)

    return result


In [11]:
def getBinStats(conn, business_id, binRange):

    sql = """
    SELECT count(business_id) as quantity, sum(checkin_count) as total_checkin FROM poi WHERE business_id IN (
        SELECT fk_poi_business_id_01 FROM has_distance WHERE distance_m >= """+str(binRange[0])+""" AND distance_m < """+str(binRange[1])+"""
                                            AND fk_poi_business_id_02 = \'"""+str(business_id)+"""\'
    )
    """

    result = executeQuery(conn, sql)

    return result

In [12]:
def getCategoryInformation(data, category):
   
    occurences = 0
    checkin = 0

     #Formato de data
    #[business_id, checkin, category]
    for item in data:
        if(item[2] == category):
            occurences = occurences + 1
            checkin = checkin + item[1]
    
    return [checkin, occurences]


In [13]:
def calculateBin(df, bin_number, w=0.5):
    
    print("executing bin:", bin_number, "\tweight:", w)

    #Criando canal de comunicação com a base de dados
    connection = connect()  

    if (connection != None):

        #Dicionário do ITDL
        scITDL = {} 
        i = 0  

        # largura do bin
        

        #centros
        for id_01, poi in df.iterrows():
            #print('calculating point:', id_01, 'id:', poi['business_id'])

            
            #Obtendo informações de categorias e checkin do poi central
            #[business_id, checkin, category]
            poi_information = getPOIInformation(connection, poi['business_id'])


            #[business_id, checkin, category, distance_m]
            bin_information = getBinInformation(connection, poi['business_id'], bin_number)
            bin_information = pd.DataFrame(bin_information, columns = ['business_id', 'category', 'checkin_count', 'distance_m'])

            #Calculando os dois parâmetros abaixo
            #sc - total de checkins no bin
            #sp - total de pois no bin
            sp = len(bin_information['business_id'].unique())
            sc = bin_information.drop_duplicates(subset = 'business_id')['checkin_count'].sum()


            #As adições são feitas baseadas nos rótulos

            #Para evitar divisão por zero
            if(sc != 0 and sp != 0):
                for center_poi in poi_information: # Para cada tki
                    for id_02, row in bin_information.iterrows(): #Para cara tkj

                        
                        #cc = all checkin o tkj
                        #cp = all occurences of tkj
                        
                        cc = bin_information[bin_information['category'] == row['category']]['checkin_count'].sum()
                        cp = bin_information[bin_information['category'] == row['category']]['category'].count()

                        #print(sc, sp, cc, cp)

                        a = (1 - (cc/sc)) #Pode gerar 0
                        u = (cp/sp)       #Pode gerar 0

                        if((a > 0) and (u > 0)):

                            A = -np.log2(a)
                            U = -np.log2(u)

                            aug = int(math.ceil((w*A) + ((1 - w)*U)))

                            #Aumentando-o pelo fator b
                            for b in range(aug):

                                scITDL[i] = {   'poi_id_center': poi['business_id'],
                                                'center_poi': center_poi[1],
                                                'poi_id_context': row['business_id'],
                                                'context_poi': row['category'], 
                                                'distance-m': row['distance_m']}
                                i = i + 1

            #break


        scITDL = pd.DataFrame.from_dict(scITDL, 'index')
        #name = './Austin/w05/Partial bins/austin-sl-tuple-n-itdl-' + str(bin_number) + 'bin-wgt'+str(w)+'-p.parquet'
        name = './Austin/w05/Partial bins/austin-sl-tuple-n-itdl-' + str(bin_number) + 'bin-wgt'+str(w)+'-p.csv'
        #scITDL.to_parquet(name, compression='brotli',  index = False)
        scITDL.to_csv(name, index=False)
        closeConnection(connection)
        
    return None

In [1]:
import csv
def calculateBin_Disco(df, bin_number, w=0.5):
    
    print("executing bin:", bin_number, "\tweight:", w)

    #Arquivo para salvar diretamente no disco
    name = './Austin/w05/Partial bins/austin-sl-tuple-n-itdl-' + str(bin_number) + 'bin-wgt'+str(w)+'-p.csv'
    csv_file = open(name, "w", newline='')
    writer = csv.writer(csv_file, delimiter=',')
    writer.writerow(["poi_id_center","center_poi","poi_id_context","context_poi","distance-m"])

    #Criando canal de comunicação com a base de dados
    connection = connect()  

    if (connection != None):



        #centros
        for id_01, poi in df.iterrows():
            #print('calculating point:', id_01, 'id:', poi['business_id'])

            
            #Obtendo informações de categorias e checkin do poi central
            #[business_id, checkin, category]
            poi_information = getPOIInformation(connection, poi['business_id'])


            #[business_id, checkin, category, distance_m]
            bin_information = getBinInformation(connection, poi['business_id'], bin_number)
            bin_information = pd.DataFrame(bin_information, columns = ['business_id', 'category', 'checkin_count', 'distance_m'])

            #Calculando os dois parâmetros abaixo
            #sc - total de checkins no bin
            #sp - total de pois no bin
            sp = len(bin_information['business_id'].unique())
            sc = bin_information.drop_duplicates(subset = 'business_id')['checkin_count'].sum()


            #As adições são feitas baseadas nos rótulos

            #Para evitar divisão por zero
            if(sc != 0 and sp != 0):
                for center_poi in poi_information: # Para cada tki
                    for id_02, row in bin_information.iterrows(): #Para cara tkj

                        
                        #cc = all checkin o tkj
                        #cp = all occurences of tkj
                        
                        cc = bin_information[bin_information['category'] == row['category']]['checkin_count'].sum()
                        cp = bin_information[bin_information['category'] == row['category']]['category'].count()

                        #print(sc, sp, cc, cp)

                        a = (1 - (cc/sc)) #Pode gerar 0
                        u = (cp/sp)       #Pode gerar 0

                        if((a > 0) and (u > 0)):

                            A = -np.log2(a)
                            U = -np.log2(u)

                            aug = int(math.ceil((w*A) + ((1 - w)*U)))

                            #Aumentando-o pelo fator b
                            for b in range(aug):

                                
                                line = [str(poi['business_id']), str(center_poi[1]), str(row['business_id']), str(row['category']), str(row['distance_m'])]
                                writer.writerow(line)

                            
            #break
        
        csv_file.close()
        closeConnection(connection)
        
    return None

# Geração do ITDL

In [14]:
#Quantidade de tuplas de vizinhos nos dados do yelp (center, context) considerando d = 100m
#df = pd.read_csv('/kaggle/input/yelpcambridge/yelpcambridge-ml.csv')
pois_file_name = './Austin/austin-ml-updated.csv'
df = pd.read_csv(pois_file_name)
print(df.shape)
df = df.dropna()
print(df.shape)
df.head()

(22399, 7)
(22399, 7)


,business_id,city,state,latitude,longitude,categories,checkin_count
0,N3_Gs3DnX4k9SgpwJxdEfw,Austin,TX,30.346169,-97.711458,"Shopping, Jewelry Repair, Appraisal Services, ...",14
1,tXvdYGvlEceDljN8gt2_3Q,Austin,TX,30.172706,-97.799920,"Barbers, Beauty & Spas",1
2,nTIhpR7MhsALPwg_Hh14EA,Austin,TX,30.326377,-97.704543,"Hotels, Hotels & Travel, Event Planning & Serv...",475
3,8XyEpVdAO0o6iVkVxkWosQ,Austin,TX,30.246465,-97.778738,"Home Services, Real Estate, Property Management",0
4,NVfOn7TdnHbaGH97CVB_Qg,Austin,TX,30.244902,-97.857409,"Chiropractors, Health & Medical",33


iteractive ITDL

In [16]:
for n in range(0, 1):
    calculateBin_Disco(df, n, 0.5)
    #break

executing bin: 0 	weight: 0.5


Parallel ITDL

In [ ]:
import multiprocessing as mp
print("Number of processors: ", mp.cpu_count())


# Step 1: Init multiprocessing.Pool()
pool = mp.Pool(int(mp.cpu_count()))

# Step 2: `pool.apply` the `howmany_within_range()`
bins = range(21, 27)
#bins = [0]
#[pool.apply(calculateBin, args=(df, n, 100, 0.3)) for n in bins]
pool.starmap(calculateBin_Disco, [(df, bin_number, 0.5) for bin_number in bins])

# Step 3: Don't forget to close
pool.close()

print('Process finished.')

Number of processors:  16
executing bin:executing bin:  1211  	weight:	weight:  0.50.5

executing bin: 13 	weight: 0.5
executing bin: 14 	weight: 0.5
executing bin: 15 	weight: 0.5
executing bin: 16 	weight: 0.5
executing bin: 17 	weight: 0.5
executing bin: 18 	weight: 0.5
0.5executing bin: 19 	weight: 
executing bin: 20 	weight: 0.5


In [19]:
import multiprocessing as mp
print("Number of processors: ", mp.cpu_count())

Number of processors:  16
